In [6]:
!pip3 install tira ir-datasets python-terrier

from tira.third_party_integrations import ensure_pyterrier_is_loaded, normalize_run, ir_datasets
import pyterrier as pt
import os
from tira.rest_api_client import Client
from glob import glob
import pandas as pd
tira = Client()
# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
ensure_pyterrier_is_loaded()

In [8]:
from trectools import TrecRun, TrecQrel, TrecEval

# The dataset: the union of the IR Anthology and the ACL Anthology
# This line creates an IRDSDataset object and registers it under the name provided as an argument.
dataset = 'ir-lab-sose-2024/ir-acl-anthology-20240504-training'
pt_dataset = pt.get_dataset('irds:' + dataset)

pt_index_path = './terrier-index'


if not os.path.exists(pt_index_path + "/data.properties"):
  # create the index, using the IterDictIndexer indexer 
  indexer = pt.index.IterDictIndexer(pt_index_path, blocks=True)

  # we give the dataset get_corpus_iter() directly to the indexer
  # while specifying the fields to index and the metadata to record
  index_ref = indexer.index(pt_dataset.get_corpus_iter(), 
                            meta=('docno',))

else:
  # if you already have the index, use it.
  index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")
index = pt.IndexFactory.of(index_ref)

ds = tira.download_dataset('ir-lab-sose-2024', dataset, truth_dataset=True)
print(ds)

def load_qrels(dataset):
    return TrecQrel(ds + '/qrels.txt')

training_qrels = load_qrels('ir-acl-anthology-20240504-training')
print("done")

Download from the Incubator: https://files.webis.de/data-in-production/data-research/tira-zenodo-dump-preparation/ir-lab-sose2024/ir-acl-anthology-20240504-inputs.zip?download=1
	This is only used for last spot checks before archival to Zenodo.


Download: 100%|██████████| 39.4M/39.4M [00:00<00:00, 72.5MiB/s]


Download finished. Extract...
Extraction finished:  /root/.tira/extracted_datasets/ir-lab-sose-2024/ir-acl-anthology-20240504-training/


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents:   0%|          | 0/126958 [00:00<?, ?it/s]/tmp/ipykernel_1323/3628429677.py:17: DeprecationWarning: specifying meta and meta_lengths in IterDictIndexer.index() is deprecated, use constructor instead
  index_ref = indexer.index(pt_dataset.get_corpus_iter(),
ir-lab-sose-2024/ir-acl-anthology-20240504-training documents:  70%|███████   | 88956/126958 [00:21<00:07, 4909.50it/s]

13:18:44.183 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (2020.mir_conference-2020.1) - further warnings are suppressed


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [00:27<00:00, 4669.92it/s] 


13:18:56.864 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 3 empty documents
/root/.tira/extracted_datasets/ir-lab-sose-2024/ir-acl-anthology-20240504-training/truth-data
done


In [41]:
def run_bm25_grid_search_run(index, output_dir, queries):
    """
        defaults: http://terrier.org/docs/current/javadoc/org/terrier/matching/models/BM25.html
        k_1 = 1.2d, k_3 = 8d, b = 0.75d
        We do not tune parameter k_3, as this parameter only impacts queries with reduntant terms.
    """
    for b in [0.1, 0.2, 0.3, 0.4]:
        for k_1 in [0.5]:
            system = f'bm25-b={b}-k_1={k_1}'
            configuration = {"bm25.b" : b, "bm25.k_1": k_1}
            run_output_dir = output_dir + '/' + system
            !rm -Rf {run_output_dir}
            !mkdir -p {run_output_dir}
            print(f'Run {system}')
            BM25 = pt.BatchRetrieve(index, wmodel="BM25", controls=configuration, verbose=True)
            run = BM25(queries)
            normalize_run(run, system, 1000).to_csv(run_output_dir + '/run.txt', sep=" ", header=False, index=False)
#, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0
#, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0

In [42]:
dataset_tira = ir_datasets.load(dataset)
queries = pt.io.read_topics(ir_datasets.topics_file(dataset), format='trecxml')
run_bm25_grid_search_run(index, 'grid-search/training', queries)

Run bm25-b=0.1-k_1=0.5


BR(BM25): 100%|██████████| 68/68 [00:01<00:00, 47.56q/s]


Run bm25-b=0.1-k_1=0.6


BR(BM25): 100%|██████████| 68/68 [00:01<00:00, 45.74q/s]


Run bm25-b=0.1-k_1=0.7


...

Run bm25-b=0.1-k_1=1.2


BR(BM25): 100%|██████████| 68/68 [00:01<00:00, 47.12q/s]


Run bm25-b=0.1-k_1=1.3


BR(BM25):   4%|▍         | 3/68 [00:00<00:02, 27.40q/s]


KeyboardInterrupt: 

In [4]:
!pip3 install trectools

from trectools import TrecRun, TrecQrel, TrecEval
from tira.rest_api_client import Client
from glob import glob
import pandas as pd
tira = Client()

def load_qrels(dataset):
    return TrecQrel(tira.download_dataset('ir-lab-sose-2024', dataset, truth_dataset=True) + '/qrels.txt')

training_qrels = load_qrels('ir-acl-anthology-20240504-test')


def evaluate_run(run_dir, qrels):
    run = TrecRun(run_dir + '/run.txt')
    trec_eval = TrecEval(run, qrels)

    return {
        'run': run.get_runid(),
        'nDCG@10': trec_eval.get_ndcg(depth=10),
        'nDCG@10 (unjudgedRemoved)': trec_eval.get_ndcg(depth=10, removeUnjudged=True),
        'MAP': trec_eval.get_map(depth=10),
        'MRR': trec_eval.get_reciprocal_rank(),
        'P@10': trec_eval.get_precision(depth=10)
    }


df = []
for r in glob('grid-search/training/bm25*'):
    df += [evaluate_run(r, training_qrels)]
df = pd.DataFrame(df)
df_sorted = df.sort_values('nDCG@10', ascending=False)
print(df_sorted)
best_configuration = df_sorted['run'].values[0]
best_b = best_configuration.split("-")[1].split("=")[1]
best_k_1 = best_configuration.split("-")[2].split("=")[1]

Download from the Incubator: https://files.webis.de/data-in-production/data-research/tira-zenodo-dump-preparation/ir-lab-sose2024/ir-acl-anthology-20240504-truth.zip?download=1
	This is only used for last spot checks before archival to Zenodo.


Download: 100%|██████████| 29.6k/29.6k [00:00<00:00, 1.67MiB/s]


Download finished. Extract...
Extraction finished:  /root/.tira/extracted_datasets/ir-lab-sose-2024/ir-acl-anthology-20240504-training/


/usr/local/lib/python3.10/dist-packages/trectools/trec_eval.py:294: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  selection = selection[~selection["rel"].isnull()].groupby("query").first().copy()
/usr/local/lib/python3.10/dist-packages/trectools/trec_eval.py:294: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  selection = selection[~selection["rel"].isnull()].groupby("query").first().copy()
/usr/local/lib/python3.10/dist-packages/trectools/trec_eval.py:294: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current 

KeyboardInterrupt: 

In [6]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25", controls= {"bm25.b" : best_b, "bm25.k_1": best_k_1})
sdm = pt.rewrite.SDM()

In [4]:
# Load the expansions
dataset_tira = ir_datasets.load(dataset)
queries = pt.io.read_topics(ir_datasets.topics_file(dataset), format='trecxml')

import pandas as pd
from pyterrier.apply import generic

def load_llm_transformer(transformer_name):
    ret = pd.read_pickle('/app/baseline-retrieval-system/llm_transformers/'+transformer_name)
    cols = [i for i in ret.columns if i not in ['qid']]
    ret = {str(i['qid']): i for _, i in ret.iterrows()}
    def __transform_df(df):
            df = df.copy()
            for col in cols:
                df['llm_expansion_' + col] = df['qid'].apply(lambda i: ret[str(i)][col])
            return df
    return generic(__transform_df)

# llm expansions with gpt
gpt_sq_fs = load_llm_transformer('gpt_sq_fs')
gpt_sq_zs = load_llm_transformer('gpt_sq_zs')

# llm expansions with llama
llama_cot = load_llm_transformer('llama_cot')
llama_sq_zs = load_llm_transformer('llama_sq_zs')

# llm expansions with flan-ul2
flan_sq_fs = load_llm_transformer('flan_sq_fs')

In [7]:
topics = pt_dataset.get_topics(variant='title')

tokeniser = pt.autoclass("org.terrier.indexing.tokenisation.Tokeniser").getTokeniser()

def pt_tokenize(text):
    return ' '.join(tokeniser.getTokens(text))

def expand_query(topic):
  ret = ' '.join([topic['query'], topic['query'], topic['query'],  topic['query'],  topic['query'], topic['llm_expansion_query']])

  # apply the tokenization
  return pt_tokenize(ret)

# we wrap this into an pyterrier transformer
# Documentation: https://pyterrier.readthedocs.io/en/latest/apply.html
pt_expand_query = pt.apply.query(expand_query)


pipeline_gpt_sq_fs = (gpt_sq_fs >> pt_expand_query) >> bm25
pipeline_gpt_sq_zs = (gpt_sq_zs >> pt_expand_query) >> bm25

pipeline_llama_cot = (llama_cot >> pt_expand_query) >> bm25
pipeline_llama_sq_zs = (llama_sq_zs >> pt_expand_query) >> bm25

pipeline_flan_sq_fs = (flan_sq_fs >> pt_expand_query) >> bm25

In [8]:
if not pt.started():
    pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])
    
topics = pt_dataset.get_topics(variant='title')

SEED=42

!pip3 install scikit-learn

from sklearn.model_selection import train_test_split

tr_va_topics, test_topics = train_test_split(topics, test_size=15, random_state=SEED)
train_topics, valid_topics =  train_test_split(tr_va_topics, test_size=5, random_state=SEED)

ltr_feats1 = bm25 >> pt.text.get_text(pt_dataset, ["text", "doc_id"]) >> (
    pt.BatchRetrieve(index, wmodel="Js_KLs")
    ** # KL Expansion
    pt.BatchRetrieve(index, wmodel="BM25", controls={"qe":"on", "qemodel" : "KL"})
    ** # ChatGPT Similiar Queries Few Shot
    pipeline_gpt_sq_fs
    ** # ChatGPT Similiar Queries Zero shot
    pipeline_gpt_sq_zs
    ** # Llama-2 Chain of Thought
    pipeline_llama_cot
    ** # Llama-2 Similiar Queries Zero shot
    pipeline_llama_sq_zs
    ** # Flan-UL2 Similiar Queries Few Shot
    pipeline_flan_sq_fs
)

# for reference, lets record the feature names here too
fnames=["Js_KLs", "KL", "gpt_sq_fs", "gpt_sq_zs", "llama_cot", "llama_sq_zs", "flan_sq_fs"]

In [10]:
from sklearn.ensemble import RandomForestRegressor
!pip3 install joblib

model = RandomForestRegressor(n_estimators=400, verbose=1, random_state=SEED, n_jobs=2)
trained_pipe = ltr_feats1 >> pt.ltr.apply_learned_model(model)
trained_pipe.fit(train_topics, pt_dataset.get_qrels())

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   12.6s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:   53.1s
[Parallel(n_jobs=2)]: Done 400 out of 400 | elapsed:  1.8min finished


In [11]:
run = trained_pipe(pt_dataset.get_topics('text'))
persist_and_normalize_run(run, system_name="RandomForestRegressor", default_output='../runs')

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.4s
[Parallel(n_jobs=2)]: Done 400 out of 400 | elapsed:    0.8s finished


The run file is normalized outside the TIRA sandbox, I will store it at "../runs".
Done. run file is stored under "../runs/run.txt".
